`Prerequisites`

To complete this exercise, you must have access to the `little_lemon` database. As an authorized user, you need to establish a connection between Python and the database via the connector API and create a cursor object

Once the connection is established, and you have a cursor object, you can select the database `little_lemon` and print the names of the tables

Tables have been used throughout the course's various sections


#### **Learning objectives:**
* Access stored procedures in a MySQL database using Python


`Scenario:`

Little lemon need to perform some tasks on daily basis, and they involve extracting data from one or more tables. 

The tasks include:
* finding the guest with maximum spending, 
* retrieving the bookings for no arrival, 
* and displaying the order status to the guests. 

In [21]:
import mysql.connector as connector
try:
    connection=connector.connect(user="root", password="")
    print('Connection established!')
except connection.Error as e:
    print(f"""
    Error code : {e.errno}|
    Error message : {e.msg}
    """)

Connection established!


In [22]:
# create cursor
cursor = connection.cursor()
cursor.execute('SHOW DATABASES')
[db[0] for db in cursor.fetchall()] # see databases

['automobile',
 'information_schema',
 'jewelrystore_db',
 'little_lemon',
 'Lucky_Shrub',
 'luckyshrub_db',
 'Mangata_Gallo',
 'mysql',
 'performance_schema',
 'restaurant',
 'soccer',
 'sys']

In [23]:
cursor.execute('USE little_lemon')
# What's our db (did this from scratch to try and remember)
connection.database

'little_lemon'

### **Task 1:**

Little Lemon is running a marketing campaign this month. They need to issue a discount coupon to the top spender on daily bases. 

Create a stored procedure `TopSpender` that can retrieve the following:
* the booking ID, 
* guest’s full name, 
* and the bill amount of the top spender at closing. 
* Call the procedure and print the results.  


`Expected Output`:
![Procedure Return 1](images/3_1.png)

In [25]:
top_spender_procedure = """
DROP PROCEDURE IF EXISTS TopSpender;
CREATE Procedure TopSpender()
BEGIN
SELECT
    O.BookingID,
    CONCAT(B.GuestFirstName, ' ', B.GuestLastName) AS CustomerName,
    O.BillAmount
FROM Orders AS O
INNER JOIN Bookings as B
USING(BookingID)
ORDER BY O.BillAmount DESC
LIMIT 1;
END
"""
# Execute
cursor.execute(top_spender_procedure, multi=True)
# call procedure
cursor.callproc('TopSpender')
# Stores results
results = next(cursor.stored_results())
print(results.column_names)
for row in results.fetchall():
    print(row)

('BookingID', 'CustomerName', 'BillAmount')
(1, 'Anna Iversen', 86)


### **Task 2:**
Help little lemon to retrieve all those bookings where the guests did not appear today. Use `NoArrival` as a name for your stored procedure. 

* Guests not appearing is indicated as a Booking not submitting any orders .. so yeah

`Expected Outuput`
![Booking No Orders](images/3_2.png)

* Now this is a good opportunity of `WHERE NOT EXISTS` type Left Join in which you want all details only in the left table and nothing that can be found in both tables
    - Now we can set the second column in the outer SELECT to None to hard_code not billamount or use the IS NULL catch on a LEFT JOIN to get only the value that didn't have a bill_amount

In [26]:
booking_no_shows = """
SELECT B.BookingID, 'None' AS BillAmount
FROM Bookings B
WHERE NOT EXISTS (
    SELECT 1
    FROM Orders O
    WHERE B.BookingID = O.BookingID
)
"""
cursor.execute(booking_no_shows)
print(cursor.column_names)
for row in cursor.fetchall():
    print(row)

('BookingID', 'BillAmount')
(6, 'None')


In [29]:
booking_no_shows_spc = """
CREATE Procedure NoArrival()
BEGIN
SELECT
    B.BookingID, O.BillAmount
FROM Bookings B
LEFT JOIN Orders O
USING(BookingID)
WHERE O.BillAmount IS NULL
ORDER BY B.BookingID;
END
"""
# Execute
cursor.execute(booking_no_shows_spc)
# call procedure
cursor.callproc('NoArrival')
# Stores results
results = next(cursor.stored_results())
print(results.column_names)
for row in results.fetchall():
    print(row)

('BookingID', 'BillAmount')
(6, None)


### **Task 3:**

It is very important for little lemon to keep track of the status of each guest’s order and display it on the screen to keep their guests informed. 

This is how they categorize the orders:
* If not assigned to any employee, the status is In Queue
* If assigned to the employees with IDs 4 and 5, the status is Preparing Order
* If assigned to the employees with IDs 1, 2, and 3, the status is Order Served


* Create a stored procedure named `OrderStatus` for the above task and call to check if everything is working.

![Case Procedure](images/3_3.png)

In [30]:
employee_assignment = """
CREATE PROCEDURE OrderStatus()
BEGIN
SELECT 
    BookingID,
    CASE
        WHEN EmployeeID in (1, 2, 3) THEN 'Order Served'
        WHEN EmployeeID in (4, 5) THEN 'Preparing Order'
        ELSE 'In Que'
    END AS Status
FROM Bookings;
END
"""
# execute 
cursor.execute(employee_assignment)
# call procedure
cursor.callproc('OrderStatus')
# store results
results = next(cursor.stored_results())
print(results.column_names)
for order_status in results:
    print(order_status)

('BookingID', 'Status')
(1, 'Order Served')
(2, 'Order Served')
(3, 'Order Served')
(4, 'Preparing Order')
(5, 'Order Served')
(6, 'Preparing Order')
